- Connect ke drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Instalasi & Setup awal

In [ ]:
import pandas as pd
import re
import nltk
import os
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, create_optimizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

2. Fungsi Preprocessing

In [ ]:
def preprocess_text(text):
    """Membersihkan teks: lowercase, hapus HTML/URL/tanda baca, stopwords, dan stemming"""
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)          # Hapus HTML tags
    text = re.sub(r'http\S+', '', text)        # Hapus URL
    text = re.sub(r'[^a-zA-Z\s]', '', text)    # Hapus non-alphabet
    text = re.sub(r'\s+', ' ', text).strip()   # Hapus spasi berlebih

    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]  # Stopwords
    words = [PorterStemmer().stem(word) for word in words]                     # Stemming
    return ' '.join(words)

# Contoh uji fungsi
print(preprocess_text("<p>I LOVEEEE this movie!!! https://example.com </p>"))  # Output: "lovee movi"

loveee movi


3. Load & Preprocess data

In [ ]:
# Load dataset
df = pd.read_csv("/content/drive/MyDrive/dataset/IMDB Dataset.csv")  # Ganti dengan path Anda
df['clean_review'] = df['review'].apply(preprocess_text)  # Apply preprocessing

# Encode label
le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'])  # positive=1, negative=0

# Split data
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['clean_review'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

print("Contoh data training:", train_texts[0][:100], "...")

Contoh data training: that kept ask mani fight scream match swear gener mayhem permeat minut comparison also stand think o ...


4. Tokenisasi dengan BERT

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenisasi data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=256)

# Konversi ke TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(1000).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(32)

print("Jumlah batch training:", len(train_dataset))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Jumlah batch training: 1250


5. Training Model

In [ ]:
# Inisialisasi model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Setup optimizer
num_train_steps = len(train_dataset) * 3
optimizer, _ = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=num_train_steps)

# Compile model
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Training
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=5
)

# Evaluasi
loss, accuracy = model.evaluate(test_dataset)
print(f"\nAkurasi test: {accuracy:.4f}")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/5
1250/1250 [==============================] - 1097s 855ms/step - loss: 0.3553 - accuracy: 0.8383 - val_loss: 0.2782 - val_accuracy: 0.8838
Epoch 2/5
1250/1250 [==============================] - 1070s 856ms/step - loss: 0.2297 - accuracy: 0.9081 - val_loss: 0.2734 - val_accuracy: 0.8896
Epoch 3/5
1250/1250 [==============================] - 1070s 856ms/step - loss: 0.1594 - accuracy: 0.9413 - val_loss: 0.2927 - val_accuracy: 0.8927
Epoch 4/5
1250/1250 [==============================] - 1070s 856ms/step - loss: 0.1327 - accuracy: 0.9531 - val_loss: 0.2927 - val_accuracy: 0.8927
Epoch 5/5
313/313 [==============================] - 86s 275ms/step - loss: 0.2927 - accuracy: 0.8927

Akurasi test: 0.8927


6. Simpan Model

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Lokasi penyimpanan
save_dir = "/content/drive/MyDrive/bert_imdb_model"
os.makedirs(save_dir, exist_ok=True)

# Simpan model & tokenizer
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Alternatif download langsung
!zip -r bert_model.zip {save_dir}
from google.colab import files
files.download("bert_model.zip")

print(f"Model disimpan di: {save_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  adding: content/drive/MyDrive/bert_imdb_model/ (stored 0%)
  adding: content/drive/MyDrive/bert_imdb_model/config.json (deflated 42%)
  adding: content/drive/MyDrive/bert_imdb_model/tf_model.h5 (deflated 8%)
  adding: content/drive/MyDrive/bert_imdb_model/tokenizer_config.json (deflated 75%)
  adding: content/drive/MyDrive/bert_imdb_model/special_tokens_map.json (deflated 42%)
  adding: content/drive/MyDrive/bert_imdb_model/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model disimpan di: /content/drive/MyDrive/bert_imdb_model


7. Fungsi Prediksi

In [ ]:
def predict_sentiment(text, model, tokenizer):
    """Prediksi sentimen dari teks input"""
    cleaned_text = preprocess_text(text)  # Preprocess konsisten
    inputs = tokenizer(
        cleaned_text,
        return_tensors="tf",
        truncation=True,
        padding=True,
        max_length=256
    )
    outputs = model(inputs)
    probs = tf.nn.softmax(outputs.logits, axis=-1)
    label = tf.argmax(probs, axis=1).numpy()[0]
    return "positive" if label == 1 else "negative"

# Contoh penggunaan
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(save_dir)
loaded_tokenizer = DistilBertTokenizer.from_pretrained(save_dir)

print(predict_sentiment("This movie sucks!", loaded_model, loaded_tokenizer))  # Output: negative

Some layers from the model checkpoint at /content/drive/MyDrive/bert_imdb_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/bert_imdb_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


negative
